In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import shutil

In [ ]:
data_path = os.path.join("..", "..", "datasets")
full_set = pd.read_csv(os.path.join(data_path, "full_set.csv"))
full_set["img_path"] = full_set["img_path"].transform(lambda x: os.path.join(data_path, x))
full_set

In [ ]:
# full_set_res получается обычным запуском скрипта, который сохраняет в res.json, а потом руками переименовал в 
# full_ser_res.json
with open(os.path.join("..", "..", "..", "personalizedImageAesthetics", "full_set_res.json")) as f:
    pred_json = json.load(f)
# pred_json

In [ ]:
preds = pd.Series(pred_json["predictScoreAll"])
preds

In [ ]:
import matplotlib.pyplot as plt
import math

def plot_images_on_grid(dataframe, img_column, save_path):
    num_images = len(dataframe)
    grid_dim = math.ceil(math.sqrt(num_images))

    fig, axes = plt.subplots(grid_dim, grid_dim, figsize=(10, 10))
    axes = axes.ravel()

    for i in range(grid_dim * grid_dim):
        ax = axes[i]
        ax.axis('off')

        if i < num_images:
            img_path = dataframe.iloc[i][img_column]
            image = plt.imread(img_path)
            ax.imshow(image)
            ax.set_title(f'Image {i + 1}')

    plt.tight_layout()
    plt.savefig(f'{save_path}.png')
    plt.show()


In [ ]:
import numpy as np

def get_thresholds(series, n):
    thresholds = np.percentile(series, np.linspace(0, 100, n+1))
    return thresholds.tolist()


In [ ]:
num_thrshlds = 50
thrshlds = np.linspace(preds.min(), preds.max(), num_thrshlds).tolist()

In [ ]:
num_thrshlds = 50
thrshlds = get_thresholds(preds, num_thrshlds)

In [ ]:
save_plts_dir = "personalizedImageAesthetics_plts"
if os.path.exists(save_plts_dir):
    shutil.rmtree(save_plts_dir)
os.mkdir(save_plts_dir)

In [ ]:
for i in range(1, num_thrshlds):
    mask = (thrshlds[i-1] < preds) & (preds <= thrshlds[i])
    plot_images_on_grid(full_set[mask][["img_path"]], "img_path", os.path.join(save_plts_dir, f"group_{i}"))

<h1> Best </h1>


In [ ]:
threshold = 0.96

In [ ]:
plt.scatter(full_set["rating"], preds)
plt.scatter(full_set["rating"][preds>threshold], preds[preds>threshold])

In [ ]:
cluster_df = pd.read_csv(os.path.join("clusters", "ViT-B32_img", "KMeans", "info.csv"))
cluster_df

In [ ]:
for cl_num in cluster_df["cl"].unique():
    cl_pred = preds[cluster_df["cl"] == cl_num]
    cl_rating = full_set[cluster_df["cl"] == cl_num]["rating"]
    
    plt.scatter(cl_rating, cl_pred)
    plt.scatter(cl_rating[cl_pred>threshold], cl_pred[cl_pred>threshold])
    plt.title(f"cluster {cl_num}")
    plt.show()

In [ ]:
full_set[["id"]][preds>threshold]

In [ ]:
save_path = f"personalizedImageAesthetics__best[{threshold}]"
if os.path.exists(save_path):
    shutil.rmtree(save_path)
os.mkdir(save_path)
    
for index, row in full_set[preds>threshold].iterrows():
    shutil.copy(
        row["img_path"],
        os.path.join(
            save_path,
            os.path.basename(row["img_path"])+f"[rating_{row['rating']}]")
    )

<h1> Worth </h1>

In [ ]:
threshold = 0.5

In [ ]:
plt.scatter(full_set["rating"], preds)
plt.scatter(full_set["rating"][preds<threshold], preds[preds<threshold])

In [ ]:
cluster_df = pd.read_csv(os.path.join("clusters", "ViT-B32_img", "KMeans", "info.csv"))
cluster_df

In [ ]:
for cl_num in cluster_df["cl"].unique():
    cl_pred = preds[cluster_df["cl"] == cl_num]
    cl_rating = full_set[cluster_df["cl"] == cl_num]["rating"]
    
    plt.scatter(cl_rating, cl_pred)
    plt.scatter(cl_rating[cl_pred<threshold], cl_pred[cl_pred<threshold])
    plt.title(f"cluster {cl_num}")
    plt.show()

In [ ]:
full_set[["id"]][preds<threshold]

In [ ]:
save_path = f"personalizedImageAesthetics_Worth[{threshold}]"
if os.path.exists(save_path):
    shutil.rmtree(save_path)
os.mkdir(save_path)
    
for index, row in full_set[preds<threshold].iterrows():
    shutil.copy(
        row["img_path"],
        os.path.join(
            save_path,
            os.path.basename(row["img_path"])+f"[{row['rating']}]")
    )